In [3]:
!pip install nbstripout
!nbstripout finetuning_personal_dataset.ipynb


Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install peft accelerate  bitsandBytes transformers  datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached bitsandbytes-0.45.5-py3-none-win_amd64.whl.metadata (5.1 kB)
     ---------------------------------------- 0.0/40.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/40.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/40.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/40.2 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.2 kB ? eta -:--:--
     ----------------------------- -------- 30.7/40.2 kB 217.9 kB/s eta 0:00:01
     ----------------------------- -------- 30.7/40.2 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 40.2/40.2 kB 174.1 kB/s eta 0:00:00
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
   ---------------------

peft stands for parameter efficient fine tuning
accelerate used to accelerate the whole process

In [ ]:
!pip install GPUtil

In [ ]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [ ]:
!pip install transformers==4.28.0

  Using cached transformers-4.28.0-py3-none-any.whl.metadata (109 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.28.0-py3-none-any.whl (7.0 MB)
Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.28.0 which is incompatible.


In [ ]:
!pip install --upgrade transformers peft
!pip install bitsandbytes accelerate

  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)
Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0
    Uninstalling transformers-4.28.0:
      Successfully uninstalled transformers-4.28.0


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig ,LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()


In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

fatal: destination path 'Fine-tuning-LLMs' already exists and is not an empty directory.


In [ ]:
train_dataset = load_dataset(
    "text",
    data_files={
        "train": [
            "/content/Fine-tuning-LLMs/data/hawaii_wf_4.txt",
            "/content/Fine-tuning-LLMs/data/hawaii_wf_2.txt"
        ]
    },
    split="train"

)

In [ ]:
train_dataset["text"][0]

'In the early morning of August 9, 2023, the officer further assisted in coordinating transportation for people who'

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id,use_fast = False , trust_remote_code = True , add_eos_token = True)
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [ ]:
tokenized_train_dataset = []
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [ ]:
tokenized_train_dataset[0]

{'input_ids': [1, 512, 278, 4688, 7250, 310, 3111, 29871, 29929, 29892, 29871, 29906, 29900, 29906, 29941, 29892, 278, 12139, 4340, 6985, 287, 297, 6615, 1218, 8608, 362, 363, 2305, 1058, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r = 8,
    lora_alpha = 64,
    target_modules = [ "q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj" ],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)
model = get_peft_model(model, config)


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
!pip uninstall -y transformers
!pip install transformers[torch] --upgrade
!pip install --upgrade accelerate datasets peft bitsandbytes

Found existing installation: transformers 4.52.3
Uninstalling transformers-4.52.3:
  Successfully uninstalled transformers-4.52.3
  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)


In [ ]:
from transformers import TrainingArguments
trainer = transformers.Trainer(
    model = model,
    train_dataset = tokenized_train_dataset,
    args = TrainingArguments( # Directly use TrainingArguments after importing it
        output_dir = "./finetunedModel",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        num_train_epochs = 3,
        learning_rate = 1e-4,
        max_steps = 20,
        bf16 = False,
        optim = "paged_adamw_8bit",
        logging_dir = "./log",
        save_strategy = "epoch",
        save_steps = 50,
        logging_steps = 10,
        report_to="none"                      # Corrected parameter name from logging_step to logging_steps
),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,mlm = False),
)
model.config.use_cache = False
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.533000
20,3.320100


TrainOutput(global_step=20, training_loss=3.4265893936157226, metrics={'train_runtime': 80.8676, 'train_samples_per_second': 0.989, 'train_steps_per_second': 0.247, 'total_flos': 90025679192064.0, 'train_loss': 3.4265893936157226, 'epoch': 0.3305785123966942})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel
# Removed the leading space before "meta-llama"
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = LlamaTokenizer.from_pretrained(base_model_id,use_fast = False , trust_remote_code = True , add_eos_token = True)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token = True
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id,use_fast = False , trust_remote_code = True , add_eos_token = True)

modelFinetuned = PeftModel.from_pretrained(base_model, "finetunedModel/checkpoint-20")

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

In [ ]:
user_question = "When did Hawaii wildfires start?"
eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"
promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
modelFinetuned.eval()
with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: When did Hawaii wildfires start? Just answer this question accurately and concisely.
sports betting sites
Hawaii Wildfires: When Did They Start?
Hawaii wildfires are a common occurrence in the state, particularly during the dry summer months. The state's unique geography and climate make it susceptible to wildfires, which can be triggered by a variety of factors such as lightning, human activity, and drought.

There have been several notable wildfires in Hawaii over the years, including:

1. 1978: The Makena Wildfire: This wildfire burned over 10,000 acres of land on the island of Maui and was one of the largest in the state's history.
2. 1983: The Waikolohe Valley Wildfire: This wildfire burned over 4,000 acres of land on the island of Oahu and was sparked by a lightning strike.
3. 2016: The Kilauea Wildfire: This wildfire burned over 2,000 acres of land on the Big Island and was sparked by a controlled burn that got out of control.
4. 2018: The Dry Creek Wildfire: This wild